# Preliminaries

<span style ="color:green;font-weight:bold"></span>

In [2]:
# import modules
import numpy as np
#... # your code here
from sklearn.datasets import load_digits

In [3]:
# base classes

class Node:
    pass

class Tree:
    def __init__(self):
        self.root = Node()
    
    def find_leaf(self, x):
        node = self.root
        while hasattr(node, "feature"):
            j = node.feature
            if x[j] <= node.threshold:
                node = node.left
            else:
                node = node.right
        return node

# Density Tree

In [57]:
class DensityTree(Tree):
    def __init__(self):
        super(DensityTree, self).__init__()
        
    def train(self, data, prior, n_min=20):
        '''
        data: the feature matrix for the digit under consideration
        prior: the prior probability of this digit
        n_min: termination criterion (don't split if a node contains fewer instances)
        '''
        self.prior = prior
        N, D = data.shape
        D_try = int(np.sqrt(D)) # number of features to consider for each split decision

        # find and remember the tree's bounding box, 
        # i.e. the lower and upper limits of the training feature set
        m, M = np.min(data, axis=0), np.max(data, axis=0)
        self.box = m.copy(), M.copy()
        
        # identify invalid features and adjust the bounding box
        # (If m[j] == M[j] for some j, the bounding box has zero volume, 
        #  causing divide-by-zero errors later on. We must exclude these
        #  features from splitting and adjust the bounding box limits 
        #  such that invalid features have no effect on the volume.)
        valid_features   = np.where(m != M)[0]
        invalid_features = np.where(m == M)[0]
        M[invalid_features] = m[invalid_features] + 1

        # initialize the root node
        self.root.data = data
        self.root.box = m.copy(), M.copy()

        # build the tree
        stack = [self.root]
        while len(stack):
            node = stack.pop()
            n = node.data.shape[0] # number of instances in present node
            if n >= n_min:
                # Call 'make_density_split_node()' with 'D_try' randomly selected 
                # indices from 'valid_features'. This turns 'node' into a split node
                # and returns the two children, which must be placed on the 'stack'.
                ... # your code here
                # compute D_try many feature_indices
                feature_indices = np.random.permutation(np.arange(D))[0:D_try]
                l, r = make_density_split_node(node, N, feature_indices)
                stack.append(l)
                stack.append(r)
            else:
                # Call 'make_density_leaf_node()' to turn 'node' into a leaf node.
                #... # your code here
                make_density_leaf_node(node, N)

    def predict(self, x):
        leaf = self.find_leaf(x)
        # return p(x | y) * p(y) if x is within the tree's bounding box 
        # and return 0 otherwise
        #return ... # your code here
        m, M = leaf.box

        if (m <= x).all() and (x <= M).all():
            return leaf.response * self.prior
        else:
            return 0

<span style ="color:green;font-weight:bold">Should boil down to the same implementation as in the sample solution.</span>

In [58]:
def make_density_split_node(node, N, feature_indices):
    '''
    node: the node to be split
    N:    the total number of training instances for the current class
    feature_indices: a numpy array of length 'D_try', containing the feature 
                     indices to be considered in the present split
    '''
    n, D = node.data.shape
    m, M = node.box
    
    # compute Volume of parent box
    V = np.prod(np.subtract(M, m))

    # find best feature j (among 'feature_indices') and best threshold t for the split
    e_min = float("inf")
    j_min, t_min = None, None
    
    for j in feature_indices:
        # Hint: For each feature considered, first remove duplicate feature values using 
        # 'np.unique()'. Describe here why this is necessary.
        
        # ANSWER: See Markdown Cell below
        
        data_unique = np.sort(np.unique(node.data[:, j]))
        # Compute candidate thresholds
        tj = [0.5*(data_unique[i] + data_unique[i+1]) for i in range(len(data_unique)-1)]
        
        # Illustration: for loop - hint: vectorized version is possible
        # TODO: vectorize
        for t in tj:
            # Compute the error
            # compute number of instances going to left and right child
            N_l = np.count_nonzero(node.data[:,j] <= t)
            N_r = n - N_l
            
            # compute volume of left and right child bounding box as a fraction of the parent bb volume
            V_l = (t - m[j])/(M[j]-m[j]) * V
            V_r = (M[j] - t)/(M[j]-m[j]) * V
            
            # use leave-one-out error formula from the exercise sheet
            err_l = N_l/(N * V_l)*(N_l/N - 2*(N_l -1)/(N-1))
            err_r = N_r/(N * V_r)*(N_r/N - 2*(N_r -1)/(N-1))
            loo_error = err_l + err_r
            
            # choose the best threshold that
            if loo_error < e_min:
                e_min = loo_error
                j_min = j
                t_min = t

    # create children
    left = Node()
    right = Node()
    
    # initialize 'left' and 'right' with the data subsets and bounding boxes
    # according to the optimal split found above
    left.data = node.data[node.data[:,j_min] <= t_min] # store data in left node -- for subsequent splits
    M_l = np.copy(M)
    M_l[j_min] = t_min
    left.box = m, M_l # store bounding box in left node
    right.data = node.data[node.data[:,j_min] > t_min]
    m_r = np.copy(m)
    m_r[j_min] = t_min
    right.box = m_r, M

    # turn the current 'node' into a split node
    # (store children and split condition)
    node.left = left
    node.right = right
    node.feature = j_min
    node.threshold = t_min

    # return the children (to be placed on the stack)
    return left, right

<span style ="color:green;font-weight:bold">Implementation same as in example solution.</span>

We need to remove duplicate feature values, because otherwise it could happen that a threshold is placed "between" two equal feature values, so it will be equal to both of them. 
This is a big problem, if these values lie on the edge of the current bounding box, because then a bounding box of empty volume will be created, which later leads to a division-by-zero-error, when we divide by the volume of that box in the computation of the error.

In [59]:
def make_density_leaf_node(node, N):
    '''
    node: the node to become a leaf
    N:    the total number of training instances for the current class
    '''
    # compute and store leaf response
    n = node.data.shape[0]
    m, M = node.box
    v = np.prod(np.subtract(M, m))
    node.response = n/(N*v)

# Decision Tree

In [60]:
class DecisionTree(Tree):
    def __init__(self):
        super(DecisionTree, self).__init__()
        
    def train(self, data, labels, n_min=20):
        '''
        data: the feature matrix for all digits
        labels: the corresponding ground-truth responses
        n_min: termination criterion (don't split if a node contains fewer instances)
        '''
        N, D = data.shape
        D_try = int(np.sqrt(D)) # how many features to consider for each split decision

        # initialize the root node
        self.root.data = data
        self.root.labels = labels
        
        stack = [self.root]
        while len(stack):
            node = stack.pop()
            n = node.data.shape[0] # number of instances in present node
            if n >= n_min and not node_is_pure(node):
                # Call 'make_decision_split_node()' with 'D_try' randomly selected 
                # feature indices. This turns 'node' into a split node
                # and returns the two children, which must be placed on the 'stack'.
                #... # your code here
                # compute D_try many feature_indices
                feature_indices = np.random.permutation(np.arange(D))[0:D_try]
                l, r = make_decision_split_node(node, feature_indices)
                stack.append(l)
                stack.append(r)
            else:
                # Call 'make_decision_leaf_node()' to turn 'node' into a leaf node.
                #... # your code here
                make_decision_leaf_node(node)
                
    def predict(self, x):
        leaf = self.find_leaf(x)
        # compute p(y | x)
        return leaf.response # your code here

<span style ="color:green;font-weight:bold">Boils down to same as example solution</span>

In [66]:
def make_decision_split_node(node, feature_indices):
    '''
    node: the node to be split
    feature_indices: a numpy array of length 'D_try', containing the feature 
                     indices to be considered in the present split
    '''
    n, D = node.data.shape

    # find best feature j (among 'feature_indices') and best threshold t for the split
    #... # your code here
    g_min = float("inf")
    j_min, t_min = None, None
    
    for j in feature_indices:        
        data_unique = np.sort(np.unique(node.data[:, j]))
        # Compute candidate thresholds
        tj = [0.5*(data_unique[i] + data_unique[i+1]) for i in range(len(data_unique)-1)]
        
        # Illustration: for loop - hint: vectorized version is possible
        # TODO: vectorize
        for t in tj:
            ## Compute the gini impurity for each child (taking the /N^2 out of the sum to speed up computations)
            
            # compute which instances are going to left and right child
            l = node.data[node.data[:,j] <= t]
            N_l = len(l)
            r = node.data[node.data[:,j] > t]
            N_r = len(r)
            
            g_l = N_l^2
            g_r = N_r^2
            for i in range(10):
                g_l -= (np.count_nonzero(l == i))^2
                g_r -= (np.count_nonzero(r == i))^2
            
            # compute sum of gini impurities
            loo_gini = g_l/N_l + g_r/N_r
            
            # choose the best threshold that
            if loo_gini < g_min:
                g_min = loo_gini
                j_min = j
                t_min = t

    # create children
    left = Node()
    right = Node()
    
    # initialize 'left' and 'right' with the data subsets and labels
    # according to the optimal split found above
    left.data = node.data[node.data[:,j_min] <= t_min] # data in left node
    left.labels = node.labels[node.data[:,j_min] <= t_min]  # corresponding labels
    right.data = node.data[node.data[:,j_min] > t_min]
    right.labels = node.labels[node.data[:,j_min] > t_min]

    # turn the current 'node' into a split node
    # (store children and split condition)
    node.left = left
    node.right = right
    node.feature = j_min
    node.threshold = t_min

    # return the children (to be placed on the stack)
    return left, right    

In [62]:
def make_decision_leaf_node(node):
    '''
    node: the node to become a leaf
    '''
    # compute and store leaf response
    node.N = len(node.labels)    
    node.response = np.argmax(np.bincount(node.labels)) # your code here

In [63]:
def node_is_pure(node):
    '''
    check if 'node' ontains only instances of the same digit
    '''
    return (len(np.unique(node.labels)) == 1) # your code here

<span style ="color:green;font-weight:bold">Again, should boil down to the same implementation.</span>

# Evaluation of Density and Decision Tree

In [6]:
# read and prepare the digits data
#... # your code here
digits = load_digits()

data = digits["data"]
images = digits["images"]
target = digits["target"]
target_names = digits["target_names"]

print(target_names)

[0 1 2 3 4 5 6 7 8 9]


### Density Tree

In [64]:
# train trees, plot training error confusion matrices, and comment on your results
#... # your code here

digits = []
density_trees = []
conf_mat = np.zeros((10, 10))

for i in range(10):
    digit = data[target == i]
    digits.append(digit)
    tree = DensityTree()
    tree.train(digit, len(digit)/len(data))
    density_trees.append(tree)
    
for i, x in enumerate(data):
    predictions = []
    for tree in density_trees:
        predictions.append(tree.predict(x))
    pred = np.array(predictions)
    res = np.argmax(pred)
    conf_mat[target[i], res] += 1


# normalize entries in each row    
conf_mat = conf_mat / np.sum(conf_mat, axis=1)
    
print(conf_mat)    


[[0.98314607 0.         0.         0.         0.01657459 0.
  0.         0.         0.         0.        ]
 [0.         0.68681319 0.00564972 0.         0.0718232  0.00549451
  0.         0.00558659 0.22988506 0.00555556]
 [0.         0.06043956 0.61016949 0.0273224  0.         0.00549451
  0.         0.         0.26436782 0.03333333]
 [0.         0.00549451 0.03389831 0.56284153 0.         0.03296703
  0.         0.03910615 0.27586207 0.06666667]
 [0.         0.         0.         0.         0.90607735 0.00549451
  0.         0.07821229 0.00574713 0.00555556]
 [0.         0.         0.         0.13114754 0.01104972 0.73626374
  0.         0.05027933 0.05172414 0.02222222]
 [0.         0.         0.         0.         0.         0.
  1.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.06629834 0.01098901
  0.         0.91620112 0.00574713 0.        ]
 [0.         0.01098901 0.         0.         0.00552486 0.01098901
  0.         0.         0.97

### Decision Tree

In [69]:
# train trees, plot training error confusion matrices, and comment on your results
#... # your code here

tree = DecisionTree()
tree.train(data, target)

conf_mat = np.zeros((10, 10))
    
for i, x in enumerate(data):
    res = tree.predict(x)
    conf_mat[target[i], res] += 1


# normalize entries in each row    
conf_mat = conf_mat / np.sum(conf_mat, axis=1)

print(conf_mat) 

[[0.90449438 0.         0.         0.00546448 0.00552486 0.04395604
  0.01657459 0.01675978 0.00574713 0.        ]
 [0.         0.89010989 0.00564972 0.05464481 0.01104972 0.
  0.00552486 0.01117318 0.01149425 0.01111111]
 [0.02808989 0.05494505 0.71751412 0.09836066 0.02209945 0.02197802
  0.         0.01117318 0.03448276 0.00555556]
 [0.00561798 0.03296703 0.07344633 0.70491803 0.00552486 0.01098901
  0.00552486 0.06145251 0.06896552 0.03888889]
 [0.02247191 0.07142857 0.01694915 0.03825137 0.6961326  0.02747253
  0.09944751 0.01675978 0.         0.01111111]
 [0.08988764 0.01648352 0.03954802 0.04371585 0.01657459 0.73076923
  0.         0.         0.02873563 0.03888889]
 [0.07865169 0.03296703 0.02824859 0.01092896 0.05524862 0.02197802
  0.74033149 0.         0.03448276 0.        ]
 [0.02247191 0.03846154 0.01694915 0.05464481 0.0441989  0.08791209
  0.01104972 0.68715084 0.00574713 0.02777778]
 [0.03370787 0.09340659 0.04519774 0.03825137 0.0718232  0.04945055
  0.0441989  0.01675

<span style ="color:green;font-weight:bold">Evaluations do not show the same exact confusion matrices and all in all our implementation seems to do a bit worse than example solution.</span>

# Density and Decision Forest

<span style ="color:green;font-weight:bold">rest not done</span>

In [7]:
class DensityForest():
    def __init__(self, n_trees):
        # create ensemble
        self.trees = [DensityTree() for i in range(n_trees)]
    
    def train(self, data, prior, n_min=20):
        for tree in self.trees:
            # train each tree, using a bootstrap sample of the data
            ... # your code here

    def predict(self, x):
        # compute the ensemble prediction
        return ... # your code here

In [35]:
class DecisionForest():
    def __init__(self, n_trees):
        # create ensemble
        self.trees = [DecisionTree() for i in range(n_trees)]
    
    def train(self, data, labels, n_min=0):
        for tree in self.trees:
            # train each tree, using a bootstrap sample of the data
            ... # your code here

    def predict(self, x):
        # compute the ensemble prediction
        return ... # your code here

# Evaluation of Density and Decision Forest

In [ ]:
# train forests (with 20 trees per forest), plot training error confusion matrices, and comment on your results
... # your code here